In [1]:
%load_ext autoreload
%autoreload 2

try:
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Vgg16'
    GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
    print(os.listdir(GOOGLE_DRIVE_PATH))
    import sys
    sys.path.append(GOOGLE_DRIVE_PATH)
except:
    pass

import time, os, torch, torchvision, random, time, math
from torch import Tensor
import torchvision
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['font.size'] = 10
import toolset as ts
from toolset import *
from toolset.utils import *
from toolset.data import *
from toolset.helper import *
from toolset.solver import *
from convolutional_networks import *
from fully_connected_networks import *


train_data, train_labels, test_data, test_labels = get_mnist_data()

if torch.cuda.is_available:
    print('Good to go!')
else:
    print('Please set GPU via Edit -> Notebook Settings.')
    



Good to go!


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
train_data = train_data.to(device)
train_labels = train_labels.to(device)
test_data = test_data.to(device)
test_labels = test_labels.to(device)

data_dict = {
    'X_train': train_data,
    'y_train': train_labels,
    'X_val': test_data,
    'y_val': test_labels,
}

cuda


In [3]:
print(train_data.shape)

torch.Size([60000, 1, 28, 28])


In [4]:
net = VggNet(
    input_dims = (1, 28, 28),
    num_filters= (32, 64, 128),
    max_pools = (0, 1, 2),
    num_FC = (10,),
    dropout = 0.5,
    weight_scale = 'kaiming',
    reg = 1e-5,
    print_params = True,
    device='cuda'
)

# net.check_loss(data_dict)

使用dropout。
参数:
	W1: torch.Size([32, 1, 3, 3])
	b1: torch.Size([32])
	gamma1: torch.Size([32])
	beta1: torch.Size([32])
	W2: torch.Size([64, 32, 3, 3])
	b2: torch.Size([64])
	gamma2: torch.Size([64])
	beta2: torch.Size([64])
	W3: torch.Size([128, 64, 3, 3])
	b3: torch.Size([128])
	gamma3: torch.Size([128])
	beta3: torch.Size([128])
	W4: torch.Size([1152, 10])
	b4: torch.Size([10])


In [5]:
reset_seed(0)
solver = Solver(net, data_dict,
                num_epochs=2, batch_size=50,
                optim_config={
                'learning_rate': 0.001,
                },
                update_rule=adam,
                verbose=True, device='cuda',print_every=100)
solver.train(time_limit=120)
# 你的模型1epoch应该能达到95%+准确率

(用时 3.22s; 迭代次数1 / 2400) loss: 2.364528
(Epoch 0 / 2, train 准确率: 19.70%, val准确率: 19.51%
更新:当前准确率大于19.51 之前的最佳准确率0.00

(用时 5.13s; 迭代次数101 / 2400) loss: 0.213891
(用时 6.81s; 迭代次数201 / 2400) loss: 0.163752
(用时 8.54s; 迭代次数301 / 2400) loss: 0.186871
(用时 10.28s; 迭代次数401 / 2400) loss: 0.024308
(用时 12.02s; 迭代次数501 / 2400) loss: 0.017234
(用时 13.78s; 迭代次数601 / 2400) loss: 0.207984
(用时 15.49s; 迭代次数701 / 2400) loss: 0.083367
(用时 17.27s; 迭代次数801 / 2400) loss: 0.053275
(用时 18.97s; 迭代次数901 / 2400) loss: 0.015938
(用时 20.62s; 迭代次数1001 / 2400) loss: 0.032132
(用时 22.31s; 迭代次数1101 / 2400) loss: 0.026353
(Epoch 1 / 2, train 准确率: 98.30%, val准确率: 98.63%
更新:当前准确率大于98.63 之前的最佳准确率19.51

(用时 24.17s; 迭代次数1201 / 2400) loss: 0.015365
(用时 25.82s; 迭代次数1301 / 2400) loss: 0.082027
(用时 27.48s; 迭代次数1401 / 2400) loss: 0.004260
(用时 29.18s; 迭代次数1501 / 2400) loss: 0.009684
(用时 30.92s; 迭代次数1601 / 2400) loss: 0.179588
(用时 32.60s; 迭代次数1701 / 2400) loss: 0.011768
(用时 34.31s; 迭代次数1801 / 2400) loss: 0.013870
(用时 36.04s; 迭代次数1901 / 

In [6]:
net.save("mnist_5_25.pth")

Saved in mnist_5_25.pth


In [12]:
net_mnist = VggNet()
net_mnist.load("mnist_5_25.pth",dtype=torch.float32,device=device)

solver_checker = Solver(net, data_dict,optim_config={
                'learning_rate': 0.001,
                },device=device)
acc = solver_checker.check_accuracy(data_dict['X_val'],data_dict['y_val'])
print(acc)

成功加载checkpoint文件: mnist_5_25.pth
0.9864667057991028
